In [ ]:
SELECT 2 + 2;

Display average age of employees separately for men and woman. Please consider all employees hired for 2003 year.

In [ ]:
SELECT Gender, AVG(DATEDIFF(YEAR, BirthDate, GETDATE())) [Avg age]
FROM HumanResources.Employee
GROUP BY Gender

Display top 3 employees’ first and last names where rate was changed more than two times. Additionally, to name please print count of rate’s changes, max rate and last date when it was changed

In [170]:
SELECT TOP 3 FirstName, LastName,
 COUNT(e.BusinessEntityID) [count of rate changes],
 MAX(ph.Rate) [max rate]
FROM HumanResources.Employee AS e
JOIN Person.Person AS p
ON e.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.EmployeePayHistory AS ph
ON e.BusinessEntityID = ph.BusinessEntityID
GROUP BY FirstName, LastName
HAVING COUNT(ph.BusinessEntityID) > 2

(3 rows affected)

Total execution time: 00:00:00.059

FirstName,LastName,count of rate changes,max rate
Wanida,Benshoof,10,150.00
David,Bradley,10,150.00
Kevin,Brown,10,150.00


Add new records to define new rate (150) with rate change date equals to current date for all employees whose Job Title contains ‘Marketing’ 


In [ ]:
UPDATE HumanResources.EmployeePayHistory
SET Rate = 150
WHERE BusinessEntityID in (
    SELECT e.BusinessEntityID
    FROM HumanResources.Employee AS e
    JOIN HumanResources.EmployeePayHistory AS h
    ON e.BusinessEntityID = h.BusinessEntityID
    WHERE JobTitle = 'Marketing' AND RateChangeDate = GETDATE()
)

Delete all records with information about rates’ changes modified ‘today’ 


In [ ]:
DELETE FROM HumanResources.EmployeePayHistory
WHERE RateChangeDate = GETDATE()

Change constraints for Rate setting it in range from 10 till 200


In [ ]:
DELETE FROM HumanResources.EmployeePayHistory
WHERE Rate < 10 or RATE > 200;

ALTER TABLE HumanResources.EmployeePayHistory
ADD CONSTRAINT Rate_Constr CHECK (Rate BETWEEN 10 AND 200);

Display information about orders with total order qty more than 10. Besides, I would like to see only orders shipped to France, Germany or United Kingdom. Please display order Id, total quantity count, order date, customer’s first name and customer’s last name and salesperson’s name (first name + last name) if it is available and territory’s name.


In [ ]:
SELECT d.SalesOrderID, OrderQty, h.OrderDate, p.FirstName, p.LastName,
pp.FirstName + ' ' + pp.[LastName] [person name], t.Name
FROM Sales.SalesOrderDetail AS d
JOIN Sales.SalesOrderHeader AS h
ON d.SalesOrderID = h.SalesOrderID
JOIN Sales.Customer AS c
ON h.CustomerID = c.CustomerID
JOIN Person.Person AS p
ON c.PersonID = p.BusinessEntityID
JOIN Sales.SalesPerson as sp
ON sp.BusinessEntityID = h.SalesPersonID
JOIN Person.Person AS pp
ON pp.BusinessEntityID = sp.BusinessEntityID
JOIN Sales.SalesTerritory AS t
ON t.TerritoryID = h.TerritoryID
WHERE OrderQty > 10 AND t.Name in ('France', 'Germany', 'United Kingdom')

Display information how much each customer (customer’s name) from US spent for shipping of his/her orders. Please consider only orders made in 2005, 2007, 2008 years. 


In [ ]:
SELECT FirstName + ' ' + LastName [Name],
    SUM(Freight) OVER (PARTITION BY h.CustomerID) [Total shipping],
    t.Name, h.OrderDate
FROM Sales.Customer AS c
JOIN Person.Person AS p
ON c.PersonID = p.BusinessEntityID
JOIN Sales.SalesOrderHeader AS h
ON c.CustomerID = h.CustomerID
JOIN Sales.SalesTerritory AS t
ON t.TerritoryID = c.TerritoryID
WHERE t.Name = 'Canada' AND YEAR(h.OrderDate) in (2005, 2007, 2008)

Display information about customer and products’ which they ordered. However, we want to see the order with minimum total amount. Please display customer’s name, product name and total amount. Please consider only orders which was shipped in times without any delays.


In [ ]:
SELECT * 
FROM
(SELECT p.FirstName, TotalDue, prod.Name,
RANK() OVER (ORDER BY TotalDue) AS [RANK]
FROM Sales.Customer AS c
JOIN Person.Person AS p
ON c.PersonID = p.BusinessEntityID
JOIN Sales.SalesOrderHeader AS h
ON c.CustomerID = h.CustomerID
JOIN Sales.SalesOrderDetail AS d 
ON d.SalesOrderID = h.SalesOrderID
JOIN Production.Product AS prod 
ON d.ProductID = prod.ProductID
WHERE h.ShipDate <= h.DueDate) t
WHERE [RANK] in (1, 2)

We want to update TaxRate in 2 times for provinces from the list:  AL, AR, AS, AZ , CA , CO


In [85]:
UPDATE Sales.SalesTaxRate
SET TaxRate = 2 * TaxRate
WHERE SalesTaxRateID in (
    SELECT tr.SalesTaxRateID
    FROM Sales.SalesTaxRate AS tr
    JOIN Person.StateProvince AS sp
    ON tr.StateProvinceID = sp.StateProvinceID
    WHERE sp.StateProvinceCode in ('AL', 'AR', 'AS', 'AZ' , 'CA' , 'CO')
)  

(2 rows affected)

Total execution time: 00:00:00.364

Display information about measurable by size products where price for them was changed more than 2 times. Want to see name, and maximum cost.


In [111]:
SELECT [Name], COUNT(ch.ProductID),
    max(ListPrice) OVER (PARTITION BY ch.ProductID) [Max price]
FROM Production.Product AS p
JOIN Production.ProductCostHistory as ch
ON p.ProductID = ch.ProductID
WHERE SizeUnitMeasureCode IS NOT NULL
GROUP BY ch.ProductID, [Name], ListPrice
HAVING COUNT(ch.ProductID) > 2;

(17 rows affected)

Total execution time: 00:00:00.066

Name,(No column name),Max price
"HL Road Frame - Red, 62",3,1431.50
"HL Road Frame - Red, 44",3,1431.50
"HL Road Frame - Red, 48",3,1431.50
"HL Road Frame - Red, 52",3,1431.50
"HL Road Frame - Red, 56",3,1431.50
"LL Road Frame - Black, 58",3,337.22
"LL Road Frame - Black, 60",3,337.22
"LL Road Frame - Black, 62",3,337.22
"LL Road Frame - Black, 44",3,337.22
"LL Road Frame - Black, 48",3,337.22


Add new column Expiration Date (default datetime more then 5 years) attribute to PersonCreditCard table.


In [113]:
ALTER TABLE Sales.PersonCreditCard
ADD [Expiration Date] varchar(255) NOT NULL DEFAULT 'More then 5 years'

Commands completed successfully.

Total execution time: 00:00:00.053

Write a script to delete PersonCreditCard column


In [119]:
DELETE [Expiration Date]
FROM Sales.PersonCreditCard
AS [Expiration Date]

(19118 rows affected)

Total execution time: 00:00:06.697

Add a script to create new table with information about potential customers (schema your name, TarasKovalyk.PotentialCustomers): CustomerId (identity column), PotentialOrderDate (nullable date time column), TerritoryId (not nullable foreign key to SalesTerritory table). Add any one record to created table (please referrer it to territory associated with United Kingdom). 



In [123]:
IF NOT EXISTS (
        SELECT *
        FROM sys.schemas
        WHERE name = N'BohdanHlovatskyi'
    )
    EXEC('CREATE SCHEMA [BohdanHlovatskyi]');
GO

IF NOT EXISTS (
    SELECT * FROM sys.objects
    WHERE object_id = OBJECT_ID(N'BohdanHlovatskyi.PotentialCustomers') 
)

BEGIN
CREATE TABLE BohdanHlovatskyi.PotentialCustomers (
    CustomerID int IDENTITY(1, 1) PRIMARY KEY,
    PotentialOrderDate date,
    TerritoryID int

    CONSTRAINT BH_TerritoryID NOT NULL FOREIGN KEY(TerritoryID) REFERENCES Sales.SalesTerritory
)
END

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.278

In [126]:
INSERT INTO BohdanHlovatskyi.PotentialCustomers (PotentialOrderDate, TerritoryID) 
VALUES (
    GETDATE(), 3
)

(1 row affected)

Total execution time: 00:00:00.157

In [127]:
SELECT * FROM BohdanHlovatskyi.PotentialCustomers

(2 rows affected)

Total execution time: 00:00:00.080

CustomerID,PotentialOrderDate,TerritoryID
1,2022-03-21,3
2,2022-03-21,3


In [133]:
IF EXISTS (
    SELECT * FROM sys.objects
    WHERE object_id = OBJECT_ID(N'BohdanHlovatskyi.PotentialCustomers') 
)
BEGIN
DROP TABLE BohdanHlovatskyi.PotentialCustomers
PRINT 'Table Sucessfully Deleted'
END
ELSE
BEGIN
PRINT 'There is no such table'
END

There is no such table

Total execution time: 00:00:00.048